In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import os

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import PIL.Image

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(device)

cuda


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed=44
seed_everything(seed) # Seed 고정

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/daconpuzzle/open/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/daconpuzzle/open/test.csv')

In [ ]:
train_np = train_df.to_numpy()
train_np

array([['TRAIN_00000', './train/TRAIN_00000.jpg', 8, ..., 9, 11, 15],
       ['TRAIN_00001', './train/TRAIN_00001.jpg', 3, ..., 12, 8, 6],
       ['TRAIN_00002', './train/TRAIN_00002.jpg', 9, ..., 5, 2, 7],
       ...,
       ['TRAIN_69997', './train/TRAIN_69997.jpg', 8, ..., 11, 9, 4],
       ['TRAIN_69998', './train/TRAIN_69998.jpg', 8, ..., 1, 16, 10],
       ['TRAIN_69999', './train/TRAIN_69999.jpg', 2, ..., 4, 11, 9]],
      dtype=object)

In [ ]:
#아래 세 함수는 test에 쓰임
def Division(a): #사진 16장으로 분할
  b = []
  for i in range(4):
    for j in range(4):
      b+=[a[:,i*128:(i+1)*128,j*128:(j+1)*128]]
  return np.array(b) #(16,3,128,128)

In [ ]:
def Con_Img(a,b): #(3,128,128)두장 입력받아서 그 사이 (3,128,64)반환
  c = np.concatenate((a[:,:,32*3:],b[:,:,:32]),axis=2).reshape(1,3,128,64)
  return c

In [ ]:
def Make_Train_Set(a): #(3,512,512)넣어서 24장 만들기
  #a는 넘파이 배열, a.shape == (3,512,512)
  b = []
  c = []
  for i in range(4):
    for j in range(3):
      b+=[a[:,i*128:i*128+128,(j+1)*128 - 32:(j+1)*128+32]]
  a_copied = a.transpose(0,2,1)
  for i in range(4):
    for j in range(3):
      c+=[a_copied[:,i*128:i*128+128,(j+1)*128 - 32:(j+1)*128+32]]
  b = np.array(b)
  c = np.array(c)
  return np.concatenate((b,c),axis=0) #shape = (24,3,128,64)

In [ ]:
def Make_Like(a,b): #a는 본래 (3,518,518)이미지, b는 순서   b에 정답넣으면 정답사진 나옴    정답은 얘가 원래 있어야 하는 위치  (1~16)
  c = Division(a)

  d0 = np.concatenate((c[b==1],c[b==2],c[b==3],c[b==4]),axis=3)
  d1 = np.concatenate((c[b==5],c[b==6],c[b==7],c[b==8]),axis=3)
  d2 = np.concatenate((c[b==9],c[b==10],c[b==11],c[b==12]),axis=3)
  d3 = np.concatenate((c[b==13],c[b==14],c[b==15],c[b==16]),axis=3)
  return np.concatenate((d0,d1,d2,d3),axis=2).reshape(3,512,512)

In [ ]:
def Make_Likethis(a,b): #이거는 여기 뭐가 와야 하는지(0~15))
  c = Division(a)

  d0 = np.concatenate((c[b[0]],c[b[1]],c[b[2]],c[b[3]]),axis=2)
  d1 = np.concatenate((c[b[4]],c[b[5]],c[b[6]],c[b[7]]),axis=2)
  d2 = np.concatenate((c[b[8]],c[b[9]],c[b[10]],c[b[11]]),axis=2)
  d3 = np.concatenate((c[b[12]],c[b[13]],c[b[14]],c[b[15]]),axis=2)
  return np.concatenate((d0,d1,d2,d3),axis=1)

In [ ]:
def Make_Wrong(a,b): #a에 이미지,b에 정답, v == 0,1,2,3로 두고 3가지 버전의 완전 틀린 퍼즐 가져오기
  aa = Make_Like(a,b)
  v = train_np[11,2:18]-1
  return Make_Likethis(aa,v)

In [ ]:
class Cnn_Model(nn.Module):  #(24,3,128,64)입력 받음.
    def __init__(self):
        super(Cnn_Model, self).__init__()
        self.conv0 = nn.Conv2d(3,256,kernel_size=5,stride=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv1 = nn.Conv2d(256,256,kernel_size=3,stride=1)
        self.conv2 = nn.Conv2d(256,128,kernel_size=3,stride=1)
        self.conv3 = nn.Conv2d(128,128,kernel_size=3,stride=1)
        self.conv4 = nn.Conv2d(128,64,kernel_size=3,stride=1)
        self.conv5 = nn.Conv2d(64,64,kernel_size=3,stride=1)
        self.fc0 = nn.Linear(64*57*25,256)
        self.fc1 = nn.Linear(256,128)
        self.fc2 = nn.Linear(128,128)
        self.fc3 = nn.Linear(128,1)

    def forward(self, x):
        x = F.relu(self.conv0(x))
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(F.relu(self.conv5(x)))
        x = torch.flatten(x,1)
        x = F.relu(self.fc0(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [ ]:
def Training(n,loss_sum):
  idx = np.random.randint(12,70000)
  hidx = str(idx)
  while len(hidx)<5:
    hidx = '0' + hidx
  try:
    img = (np.array(PIL.Image.open(f'/content/drive/MyDrive/daconpuzzle/open/train/TRAIN_{hidx}.jpg'))/255)*2-1
  except:
    hidx = '00000'
    idx = 0
    img = (np.array(PIL.Image.open(f'/content/drive/MyDrive/daconpuzzle/open/train/TRAIN_{hidx}.jpg'))/255)*2-1

  img = np.transpose(img,(2,0,1))
  cor_img = Make_Like(img,train_np[idx,2:18])
  img_set = Make_Train_Set(cor_img)

  wor_img = Make_Wrong(img,train_np[idx,2:18])
  wor_img = Make_Train_Set(wor_img)
  img_set = np.concatenate((img_set,wor_img),axis=0)

  img_set = torch.tensor(img_set, dtype=torch.float32)
  img_set = img_set.to(device)
  output = model(img_set)
  answer = np.array([1]*24+[0]*24).reshape(-1,1)
  answer = torch.tensor(answer, dtype=torch.float32)
  answer = answer.to(device)
  optimizer.zero_grad()
  loss = criter(output, answer)
  loss_sum = loss_sum+loss
  if n % 100 == 0:
    print('------',n,'-------')
    print(loss_sum)
    loss_sum = 0
  loss.backward()
  optimizer.step()

  return loss_sum

In [ ]:
model = Cnn_Model().to(device)
optimizer = optim.Adam(model.parameters())
criter = nn.BCELoss()

훈련, 저장, 불러오기 전부다 실행하지 말고 해야하는 것만 하기

In [ ]:
model = torch.load('/content/drive/MyDrive/daconpuzzle/open/model_weights_3.pth',map_location=device)

In [ ]:
loss_sum = 0
for j in range(10):
  for i in range(300): #18000개 학습 함.
    loss_sum = Training(j*300+i,loss_sum)
  torch.save(model, '/content/drive/MyDrive/daconpuzzle/open/model_weights_3.pth')

In [ ]:
#함부로 돌리면 안됨 torch.save(model, '/content/drive/MyDrive/daconpuzzle/open/model_weights_2.pth')

In [ ]:
def FF(a,b):
  aa = Con_Img(a,b)
  aa = torch.tensor(aa, dtype=torch.float32)
  aa = aa.to(device)
  return model(aa)

def GG(a,b):
  aa = Con_Img(np.rot90(a,1,axes=(1,2)),np.rot90(b,1,axes=(1,2)))
  aa = torch.tensor(aa, dtype=torch.float32)
  aa = aa.to(device)
  return model(aa)

In [ ]:
deg = 0.9
#테스트 데이터로 사진 뽑기 코드
model.eval()
idx = 0

wc = 0
patience = 3
while idx < 0+20:
  hidx = str(idx)

  while len(hidx)<5:
    hidx = '0' + hidx
  try:
    img = (np.array(PIL.Image.open(f'/content/drive/MyDrive/daconpuzzle/open/test/TEST_{hidx}.jpg'))/255)*2-1
  except:
    continue

  img = np.transpose(img,(2,0,1))
  answer = img
  img = Division(img)

  mt = (np.ones((7,7))*-1).astype(np.int16)
  lst = np.arange(16)

  if wc>0:
    np.random.shuffle(lst)

  x = 3   #현재의 x,y위치
  y = 3
  h = 1

  mt[x,y] = lst[0]  #matrix
  lst = np.delete(lst,0) #list
  y = y+h

  count = 0
  wc = 0
  while count<3:
    for i in lst:
      if h == 1:
          if FF(img[mt[x,y-h]],img[i])>deg:
              mt[x,y] = i
              lst = np.delete(lst,lst == i)
              y = y+h
              count = count + 1
              break
      elif h == -1:
          if FF(img[i],img[mt[x,y-h]]) > deg:
              mt[x,y] = i
              lst = np.delete(lst,lst == i)
              y = y+h
              count = count + 1
              break
      if i == lst[-1] and h == 1:
        h = -1
        y = 3
        y = y + h
      elif i == lst[-1] and h == -1:
        mt[x,y] = lst[0]
        lst = np.delete(lst,0)
        y = y+h
        count = count + 1
        wc = wc+1

  if h == 1:
    start_y = 4
  else:
    start_y = y - h + 1

  x_count = 0
  count = 0

  hx = 1
  h = 1
  y = start_y + h #F에 넣을 때 y-h로 시작부터 통일하기 위해
  ct = 0

  while x_count < 3:
    for i in lst:
        if hx == 1:
          if GG(img[mt[x,start_y]],img[i]) > deg:
              x = x + hx
              mt[x,start_y] = i
              lst = np.delete(lst,lst == i)
              x_count = x_count + 1
              break
        if hx == -1:
          if GG(img[i],img[mt[x,start_y]]) > deg:
              x = x + hx
              mt[x,start_y] = i
              lst = np.delete(lst,lst == i)
              x_count = x_count + 1
              break
        if i == lst[-1] and hx == 1:
          hx = -1
          x = 3
          ct = 1
          break
        if i == lst[-1] and hx == -1:
          x = x + hx
          mt[x,start_y] = lst[0]
          lst = np.delete(lst,0)
          x_count = x_count + 1
          wc = wc+1
          break
    if ct == 1:
      ct = 0
      continue
    count = 0
    y = start_y + h
    while count < 2:
        for i in lst:
          if hx == 1:
            if FF(img[mt[x,y-h]],img[i])+GG(img[mt[x-hx,y]],img[i]) > 2*deg:
                mt[x,y] = i
                lst = np.delete(lst,lst == i)
                y = y + h
                count = count + 1
                break
          if hx == -1:
            if FF(img[mt[x,y-h]],img[i])+GG(img[i],img[mt[x-hx,y]]) > 2*deg:
                mt[x,y] = i
                lst = np.delete(lst,lst == i)
                y = y + h
                count = count + 1
                break
          if i == lst[-1]:
              mt[x,y] = lst[0]
              lst = np.delete(lst,0)
              y = y + h
              count = count + 1
              wc = wc+1
              break
    for i in lst:
          if hx == 1:
            if FF(img[i],img[mt[x,start_y]])+GG(img[mt[x-hx,start_y-1]],img[i]) > 2*deg:
                mt[x,start_y-1] = i
                lst = np.delete(lst,lst == i)
                y = y + h
                count = count + 1
                break
          if hx == -1:
              if FF(img[i],img[mt[x,start_y]])+GG(img[i],img[mt[x-hx,start_y-1]]) > 2*deg:
                mt[x,start_y-1] = i
                lst = np.delete(lst,lst == i)
                y = y + h
                count = count + 1
                break
          if i == lst[-1]:
              mt[x,start_y-1] = lst[0]
              lst = np.delete(lst,0)
              y = y + h
              count = count + 1
              wc = wc+1
              break

  if wc>1 and patience >0 :
    patience -= 1
    continue
  print(mt)
  print(wc)
  print(patience)

  fmt = []
  for i in range(7):
    for j in range(7):
      if not mt[i,j] == -1:
        fmt += [mt[i,j]]

  fmt = np.array(fmt)

  answer = Make_Likethis(answer,fmt)
  answer = (np.transpose(answer,(1,2,0))+1)/2
  plt.imshow(answer)
  plt.show()
  idx += 1
  patience = 3
  wc = 0

In [ ]:
nono = []

In [ ]:
model.eval()
for iii in range(12,599):#12까지 함
  print('iii :',iii)
  submit = pd.read_csv('/content/drive/MyDrive/daconpuzzle/open/sample_submission.csv').to_numpy()
  deg = 0.9
  idx = iii*100

  wc = 0
  patience = 20
  while idx < (iii+1)*100:
    hidx = str(idx)

    while len(hidx)<5:
      hidx = '0' + hidx
    try:
      img = (np.array(PIL.Image.open(f'/content/drive/MyDrive/daconpuzzle/open/test/TEST_{hidx}.jpg'))/255)*2-1
    except:
      nono+=[idx]
      idx+=1
      continue

    img = np.transpose(img,(2,0,1))
    answer = img
    img = Division(img)

    mt = (np.ones((7,7))*-1).astype(np.int16)
    lst = np.arange(16)

    if wc>0:
      np.random.shuffle(lst)

    x = 3   #현재의 x,y위치
    y = 3
    h = 1

    mt[x,y] = lst[0]  #matrix
    lst = np.delete(lst,0) #list
    y = y+h

    count = 0
    wc = 0
    while count<3:
      for i in lst:
        if h == 1:
            if FF(img[mt[x,y-h]],img[i])>deg:
                mt[x,y] = i
                lst = np.delete(lst,lst == i)
                y = y+h
                count = count + 1
                break
        elif h == -1:
            if FF(img[i],img[mt[x,y-h]]) > deg:
                mt[x,y] = i
                lst = np.delete(lst,lst == i)
                y = y+h
                count = count + 1
                break
        if i == lst[-1] and h == 1:
          h = -1
          y = 3
          y = y + h
        elif i == lst[-1] and h == -1:
          mt[x,y] = lst[0]
          lst = np.delete(lst,0)
          y = y+h
          count = count + 1
          wc = wc+1

    if h == 1:
      start_y = 4
    else:
      start_y = y - h + 1

    x_count = 0
    count = 0

    hx = 1
    h = 1
    y = start_y + h #F에 넣을 때 y-h로 시작부터 통일하기 위해
    ct = 0

    while x_count < 3:
      for i in lst:
          if hx == 1:
            if GG(img[mt[x,start_y]],img[i]) > deg:
                x = x + hx
                mt[x,start_y] = i
                lst = np.delete(lst,lst == i)
                x_count = x_count + 1
                break
          if hx == -1:
            if GG(img[i],img[mt[x,start_y]]) > deg:
                x = x + hx
                mt[x,start_y] = i
                lst = np.delete(lst,lst == i)
                x_count = x_count + 1
                break
          if i == lst[-1] and hx == 1:
            hx = -1
            x = 3
            ct = 1
            break
          if i == lst[-1] and hx == -1:
            x = x + hx
            mt[x,start_y] = lst[0]
            lst = np.delete(lst,0)
            x_count = x_count + 1
            wc = wc+1
            break
      if ct == 1:
        ct = 0
        continue
      count = 0
      y = start_y + h
      while count < 2:
          for i in lst:
            if hx == 1:
              if FF(img[mt[x,y-h]],img[i])+GG(img[mt[x-hx,y]],img[i]) > 2*deg:
                  mt[x,y] = i
                  lst = np.delete(lst,lst == i)
                  y = y + h
                  count = count + 1
                  break
            if hx == -1:
              if FF(img[mt[x,y-h]],img[i])+GG(img[i],img[mt[x-hx,y]]) > 2*deg:
                  mt[x,y] = i
                  lst = np.delete(lst,lst == i)
                  y = y + h
                  count = count + 1
                  break
            if i == lst[-1]:
                mt[x,y] = lst[0]
                lst = np.delete(lst,0)
                y = y + h
                count = count + 1
                wc = wc+1
                break
      for i in lst:
            if hx == 1:
              if FF(img[i],img[mt[x,start_y]])+GG(img[mt[x-hx,start_y-1]],img[i]) > 2*deg:
                  mt[x,start_y-1] = i
                  lst = np.delete(lst,lst == i)
                  y = y + h
                  count = count + 1
                  break
            if hx == -1:
                if FF(img[i],img[mt[x,start_y]])+GG(img[i],img[mt[x-hx,start_y-1]]) > 2*deg:
                  mt[x,start_y-1] = i
                  lst = np.delete(lst,lst == i)
                  y = y + h
                  count = count + 1
                  break
            if i == lst[-1]:
                mt[x,start_y-1] = lst[0]
                lst = np.delete(lst,0)
                y = y + h
                count = count + 1
                wc = wc+1
                break

    if wc>2 and patience >0 :
      patience -= 1
      continue

    fmt = []
    for i in range(7):
      for j in range(7):
        if not mt[i,j] == -1:
          fmt += [mt[i,j]]
    idx += 1
    patience = 20
    wc = 0
    answer = np.arange(16)
    for i in range(16):
      answer[fmt[i]] = i
    submit[idx,1:] = answer+1

  submit = pd.DataFrame(submit,columns = ['ID','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16'])
  submit.to_csv('/content/drive/MyDrive/daconpuzzle/open/sample_submission.csv',index = None)